In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

# The Chinook music store
In order to practice with databases we will crawl into the skin of a data analyst at a fictional music store called Chinook.
(Now I am a member of Chinook)
Our database has several tables to work with we will make queries using SQL in order to guide the decisions for important business choices here. For your convenience this is the database scheme:
![image](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view"); 

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql  -- #These are the employees, let's see who I could be.
SELECT first_name || ' ' || last_name name,
       title 
  FROM employee
     -- #(Robert King is a great name, you can call me that way ;-)

 * sqlite:///chinook.db
Done.


name,title
Andrew Adams,General Manager
Nancy Edwards,Sales Manager
Jane Peacock,Sales Support Agent
Margaret Park,Sales Support Agent
Steve Johnson,Sales Support Agent
Michael Mitchell,IT Manager
Robert King,IT Staff
Laura Callahan,IT Staff


## Which new records to purchase for the USA market?
There are four new records we could add to our store, we only know the genre of the new albums. Let's find out which genre sells best in the USA and in the end we can select three of the four albums for the Chinook store. 

In [4]:
%%sql -- #First we should see which records are from USA
SELECT billing_country
  FROM invoice
 LIMIT 5;

 * sqlite:///chinook.db
Done.


billing_country
USA
Canada
France
USA
USA


In [5]:
%%sql -- #Hence we need to link the invoice table to tracks to filter the tracks from USA only!
CREATE VIEW us_track AS
SELECT t.name track_name,
       i.billing_country country, 
       t.genre_id
  FROM track t 
  JOIN invoice_line il ON il.track_id = t.track_id
  JOIN invoice i ON i.invoice_id = il.invoice_id
 WHERE billing_country = 'USA'

 * sqlite:///chinook.db
(sqlite3.OperationalError) table us_track already exists
[SQL: --Hence we need to link the invoice table to tracks to filter the tracks from USA only!
CREATE VIEW us_track AS
SELECT t.name track_name,
       i.billing_country country, 
       t.genre_id
  FROM track t 
  JOIN invoice_line il ON il.track_id = t.track_id
  JOIN invoice i ON i.invoice_id = il.invoice_id
 WHERE billing_country = 'USA']
(Background on this error at: http://sqlalche.me/e/e3q8)


In [6]:
%%sql -- #Now we need to aggegrate the tracks in the view on genre.

    SELECT g.name genre,
           COUNT(ut.track_name) tracks_per_genre,
           COUNT(ut.track_name) * 100.0 / SUM(COUNT(ut.track_name)) OVER () AS Percentage
      FROM us_track ut
      JOIN genre g ON g.genre_id = ut.genre_id
     GROUP BY 1
     ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_per_genre,Percentage
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.4253092293054235
Alternative,35,3.330161750713606
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


## Three tracks
The genres of our new tracks are: Punk, Pop, Hip-Hop and Blues. From the above query it is clear that we should add more to the Alternative & Punk category which seems to score well, even when you would remove the alternative part I am sure there will be enough Punk left. Blues is also doing well. Whereas Hip_hop and Pop are very close to each other. With pop being the close winner, therefore we go for Punk, Blues and Pop.

The corresponding artists for those genres are: Red Tone, Slim Jim Bites and Meteor and the Girls.

## Sales team performance
At Chinook all customers are paired with sales agents from their first purchase onwards. We are able to find out which sales support agent has sold the most (in $). Let's find a candidate for employee of the year.

In [7]:
%%sql
SELECT * FROM employee

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [8]:
%%sql -- #Firstly, we will select all employees with sales titles. Then we are going to link that to the total $ amount.
WITH sales_support_employee AS
    (
    SELECT * FROM employee
     WHERE title LIKE "%Support%")

SELECT sse.first_name || ' ' || sse.last_name name,
       SUM(i.total) || '$' total_sold
  FROM sales_support_employee sse
  JOIN customer c ON c.support_rep_id = sse.employee_id
  JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY 1
 ORDER BY 2 DESC;  

 * sqlite:///chinook.db
Done.


name,total_sold
Jane Peacock,1731.51$
Margaret Park,1584.0$
Steve Johnson,1393.92$


## Jane Peacock
What's in a name for sales! It's not about age *per se*, even though Jane is the youngest, Margaret is the oldest and she is still performing better than Steve. 
Nevertheless, it could very well be possible that Steve is not assigned as many customers in our system. 

## Collecting data from different countries
In the next queries we are going to sum up the data per country. Especially interesting values are:
* the total number of customers 
* the total value 
* average value of sales per customer 
* average order value. 

Of course if a country only has one customer, it is less interesting hence those numbers will be at the bottom of the table. 

In [9]:
%%sql -- #We will need four columns: country(customer table), customer_id, invoice_id, total.
      -- #We are able to find all these in the invoice table!
WITH country_and_other AS
    (
    SELECT i.*,
            CASE
            WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country
      FROM customer c
      JOIN invoice i ON i.customer_id = c.customer_id
    ),
    
    country_sort AS
    (
     SELECT country,
            COUNT(DISTINCT customer_id) total_customers,
            SUM(total) total_sales,
            SUM(total)/COUNT(DISTINCT customer_id) avg_customer_value,
            SUM(total)/COUNT(invoice_id) avg_order_value
       FROM country_and_other
      GROUP BY 1
    )
    
SELECT country,
       total_customers,
       total_sales,
       avg_customer_value,
       avg_order_value 
  FROM 
    (
    SELECT
        cs.*,
        CASE
            WHEN cs.country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_sort cs
   )
 ORDER BY sort, total_customers DESC;



 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_customer_value,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
United Kingdom,3,245.52,81.84,8.768571428571429
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


## Chances
From this table we can see that the countries to focus on are USA, Brazil and Czech Republic. USA and Brazil have a large enough customer base. Whereas Czech Republic might have a special interest regarding our shop, there might be more big customers there.

# Are you single? No I'm Album.
In our next query we are going to assess the possibility of not buying whole albums for our store. For instance, when there are albums with two popular tracks, whereas the rest does not get bought, it makes sense to consider buying less albums from the record label.

There are two ways of buying an album from Chinook. One way is to add the album, but then you are not able to add other tracks to that purchase. The only exception to this is when all tracks are selected manually. Then one is able to add other tracks. Fortunately the company has confirmed that this does not happen very often. 

Therefore we need to select invoices and check them if they have album purchases or not. *One catch* there are a lot of albums with only one or two tracks. Those cannot reasonably be called *album* and we should exclude those from our investigation. We have chosen to focus on albums only containing more than five tracks.

In [12]:
%%sql   -- Let's see how many tracks there are per album.  
WITH tracks_per_album AS
    (
    SELECT album_id,
      COUNT(track_id) tracks   
      FROM track
     GROUP BY 1
    )
    
SELECT tracks,
       COUNT(album_id)
  FROM tracks_per_album
 GROUP BY 1

 * sqlite:///chinook.db
Done.


tracks,COUNT(album_id)
1,82
2,8
3,3
4,2
5,2
6,3
7,8
8,14
9,15
10,27


In [24]:
%%sql
WITH albums_invoice AS
    (
    SELECT il.invoice_id,
       COUNT(DISTINCT t.album_id) different_albums,
       COUNT(DISTINCT t.track_id) amount_tracks
      FROM invoice_line il
      JOIN track t ON t.track_id = il.track_id
     GROUP BY 1
    ),
    
    albums_invoice_def AS
    (
    SELECT *,
       CASE
       WHEN different_albums == 1 AND amount_tracks > 5 THEN "Album" 
       ELSE "Singles" -- #The line above filters for invoices containing only 1 album id and at least 6 tracks
       END AS album_singles
        FROM albums_invoice
    )
    
SELECT album_singles,
       COUNT(invoice_id) count,
       COUNT(invoice_id) * 100.0 / SUM(COUNT(invoice_id)) OVER () AS percentage
  FROM albums_invoice_def
 GROUP BY 1

 * sqlite:///chinook.db
Done.


album_singles,count,percentage
Album,132,21.498371335504885
Singles,482,78.50162866449512


### Results
There is definitely no reason to stop the album sales as they make up 21% of all sales. It could be a little less because some customers might have bought six single tracks from one album, but this amount is negligible.